In [83]:
# Active l'affichage de résultats multiples par cellule
# %matplotlib inline (pour des graphes)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [84]:
from pandas import options

# arrondi des montants (dont plafonds de loyers, au centime d'euro le plus proche)
options.display.float_format = '{:.2f}'.format
NB_DECIMALES = 2

In [85]:
situation_18yo_moving_away = {
    "id": "18yo-moving-away",
    "description": "Un jeune de 18 ans déménage pour des études à l'université",
    "answers": {
        "statut-professionnel": "etudiant",  # activite
        "situation-professionnelle": "sans-emploi",  # dispatchSituationProfessionnelle
        "etudiant-mobilite": "parcoursup-nouvelle-region",  # dispatchEtudiantMobilite
        "boursier": True,  # boursier
        "date-naissance": "2007-03-01",  # date_naissance
        "handicap": False,  # handicap
        "statut-marital": "celibataire",  # statut_marital
        "code-postal-nouvelle-ville": "75101",  # depcom
        "situation-logement": "locataire",  # dispatchSituationLogement
        "type-logement": "logement-meuble",  # dispatchTypeLogement
        "logement-conventionne": True,  # logement_conventionne
        "colocation": False,  # coloc
        "logement-parente-proprietaire": False,  # proprietaire_proche_famille
        "nombre-personnes-logement": 1,  # 🔥 exclude: True
        "loyer-montant-mensuel": 700,  # loyer
        "loyer-montant-charges": 100,  # charges_locatives
        "loyer-difficile-payer": True,  # exclude: True
        "type-revenus": [
            "aucun-autres-revenus"
        ],
        "confirmation-end": [
            "confirmation-end-oui"
        ]
    },
    "questionsToApi": [
        "locapass-eligibilite",
        "mobilite-master-1",
        "mobilite-parcoursup",
        "aide-personnalisee-logement",
        "garantie-visale-eligibilite",
        "garantie-visale"
    ],
    "results": {
        "locapass": 1200,
        "locapass-eligibilite": True,
        "mobilite-master-1": 0,
        "mobilite-master-1-eligibilite": False,
        "mobilite-parcoursup": 500,
        "mobilite-parcoursup-eligibilite": True,
        "aide-personnalisee-logement": 327,
        "aide-personnalisee-logement-eligibilite": True,
        "garantie-visale": 800,
        "garantie-visale-eligibilite": True
    }
}

to_test = situation_18yo_moving_away

In [86]:
from notebooks.utils_mapping_simulateur import format_to_openfisca_json
from datetime import datetime
from dateutil.relativedelta import relativedelta


today = datetime.now()
today_year_month = today.strftime("%Y-%m")
year = today.strftime("%Y")
period = today_year_month  # par exemple : '2025-04'

last_month = today - relativedelta(months=1)
period_last_month  = last_month.strftime("%Y-%m")

situation_apl = format_to_openfisca_json(to_test, period)

In [87]:
from openfisca_core.simulation_builder import SimulationBuilder
from openfisca_france import FranceTaxBenefitSystem


tax_benefit_system = FranceTaxBenefitSystem()

sb = SimulationBuilder()
simulation = sb.build_from_entities(tax_benefit_system, situation_apl)

apl_variables = ['apl', 'logement_conventionne', 'aide_logement_montant', 'aide_logement_montant_brut', 'zone_apl']
# apl_variables_entities = ['familles', 'menages', 'familles', 'familles', 'menages']
# apl_variables_groupes = ['famille_1', 'menage_1', 'famille_1', 'famille_1', 'menage_1']

for variable_name in apl_variables: 
    print(f"{variable_name}: {simulation.calculate(variable_name, period)}")

parametres_secteur_locatif_period = tax_benefit_system.parameters(period).prestations_sociales.aides_logement.allocations_logement.locatif

# Pour mémoire, à partir d'ici, tous les simulation.calculate d'une variable impliquée dans le calcul 'apl'
# fera appel au cache des valeurs de la simulation (n'impliquera pas de re-calcul)

apl: [181.65]
logement_conventionne: [ True]
aide_logement_montant: [181.65]
aide_logement_montant_brut: [236.66]
zone_apl: ['zone_1']


In [88]:
en_couple = simulation.calculate('en_couple', period)[0]
celibataire = ~en_couple
label_situation_maritale = 'en couple' if en_couple else 'célibataire'
print(f"Situation maritale : {label_situation_maritale}")

al_nb_personnes_a_charge = simulation.calculate('al_nb_personnes_a_charge', period)[0]
print(f"Nombre de personnes à chage au sens des aides au logement : {al_nb_personnes_a_charge}")

au_moins_une_personne_a_charge = al_nb_personnes_a_charge > 0

Situation maritale : célibataire
Nombre de personnes à chage au sens des aides au logement : 0


## L - Loyer pris en compte

```

L est le loyer mensuel réel pris en compte dans la limite d’un plafond variable en fonction de trois zones géographiques et du nombre de personnes à charge.

```

In [89]:
from pandas import DataFrame, Index
from utils_display import affiche_resultat, colorie_reference

In [90]:
from openfisca_france.model.prestations.aides_logement import TypesZoneApl, TypesStatutOccupationLogement


zone_apl = simulation.calculate('zone_apl', period)[0]
id_zone_apl_applicable = TypesZoneApl.names[zone_apl]
print(f"Quelle est la zone APL du logement ? '{id_zone_apl_applicable}'")


print(f"\nDans le cas d'un logement en chambre, d'une location meublée, d'une colocation ou d'une sous-location, le loyer L pris en compte est réduit.")

logement_chambre = simulation.calculate('logement_chambre', period)[0]
print(f"→ Le logement est-il une chambre ? {logement_chambre}")

statut_occupation_logement = simulation.calculate('statut_occupation_logement', period)[0]
location_meuble = statut_occupation_logement == TypesStatutOccupationLogement.locataire_meuble
print(f"→ Le logement est-il une location de meublé ? {location_meuble}")

coloc = simulation.calculate('coloc', period)[0]
print(f"→ Le logement est-il une colocation ? {coloc}")

print(f"→ Le logement est-il une sous-location ? [X] TODO")


Quelle est la zone APL du logement ? 'zone_1'

Dans le cas d'un logement en chambre, d'une location meublée, d'une colocation ou d'une sous-location, le loyer L pris en compte est réduit.
→ Le logement est-il une chambre ? False
→ Le logement est-il une location de meublé ? False
→ Le logement est-il une colocation ? False
→ Le logement est-il une sous-location ? [X] TODO


In [91]:
loyer = simulation.calculate('loyer', period)[0]

# aide_logement_loyer_plafond (intégrant la zone_apl, la situation familiale, la colocation et le logement chambre)
# aide_logement_loyer_reel (intégrant les coefficients  de chambre et logement meublé)
aide_logement_loyer_retenu = simulation.calculate('aide_logement_loyer_retenu', period)[0]
L_calcule = round(aide_logement_loyer_retenu, NB_DECIMALES)

print(f"\nPour un loyer déclaré de {loyer} €, le montant calculé est :\nL = {L_calcule:.2f} €")

loyer_plafond_toute_situation = None


Pour un loyer déclaré de 700.0 €, le montant calculé est :
L = 329.71 €


In [92]:
id_zone_1 = TypesZoneApl.names[1]
id_zone_2 = TypesZoneApl.names[2]
id_zone_3 = TypesZoneApl.names[3]

parametres_locatif_zone_1 = parametres_secteur_locatif_period.formule.l_plafonds_loyers.par_zone[id_zone_1]
parametres_locatif_zone_2 = parametres_secteur_locatif_period.formule.l_plafonds_loyers.par_zone[id_zone_2]
parametres_locatif_zone_3 = parametres_secteur_locatif_period.formule.l_plafonds_loyers.par_zone[id_zone_3]

plafond_loyer_period_zone = parametres_secteur_locatif_period.formule.l_plafonds_loyers.par_zone[id_zone_apl_applicable]

### Montant en euros du plafond de loyer si logement chambre (quelle que soit la situation familiale)

In [93]:
# TABLEAU LOGEMENT CHAMBRE
# TODO tableau complémentaire personne âgée ou handicapée adulte hébergée à titre onéreux chez des particuliers

affiche_resultat("logement chambre", "Le logement est-il une chambre ?", logement_chambre, logement_chambre)

if logement_chambre:
    coef_chambre = parametres_secteur_locatif_period.formule.l_plafonds_loyers.coef_chambre_et_colocation.coef_chambre
    plafond_loyer_applicable_chambre = coef_chambre * plafond_loyer_period_zone.personnes_seules
    loyer_plafond_toute_situation = plafond_loyer_applicable_chambre

    data_parametres_chambre = {
        ('Logement chambre (cas général)', 'Montant'): [
            coef_chambre * parametres_locatif_zone_1.personnes_seules,
            coef_chambre * parametres_locatif_zone_2.personnes_seules,
            coef_chambre * parametres_locatif_zone_3.personnes_seules
            ]
    }

    index = Index([id_zone_1, id_zone_2, id_zone_3], name='Zone')
    df_parametres_chambre = DataFrame(data_parametres_chambre, index=index)
    df_parametres_chambre.style.format(precision=NB_DECIMALES).map(colorie_reference, reference=plafond_loyer_applicable_chambre)


❌ Le logement est-il une chambre ?
logement chambre : 0.00


### Montant en euros du plafond de loyer si colocation (quelle que soit la situation familiale)

TODO - hors périmètre simulateur déménagement aides-simplifiées à ce stade

### Montant du plafond de loyer selon situation familiale (hors logement chambre, hors colocation, hors sous-location)

In [94]:
# TABLEAU SANS PERSONNE À CHARGE

data_parametres_sans_personne_a_charge = {
    'Zone': [id_zone_1, id_zone_2, id_zone_3],
    'Personne seule': [parametres_locatif_zone_1.personnes_seules, parametres_locatif_zone_2.personnes_seules, parametres_locatif_zone_3.personnes_seules],
    'Couple': [parametres_locatif_zone_1.couples, parametres_locatif_zone_2.couples, parametres_locatif_zone_3.couples]
}

df_parametres_sans_personne_a_charge = DataFrame(data_parametres_sans_personne_a_charge)

# Définir la colonne 'Zone' comme index (optionnel, pour correspondre à l'exemple de la brochure APL)
df_parametres_sans_personne_a_charge = df_parametres_sans_personne_a_charge.set_index('Zone')

In [95]:
# TABLEAU AVEC PERSONNE À CHARGE

def get_plafond_loyer(zone_parameters, nb_personnes_a_charge):
    premier_enfant = zone_parameters.un_enfant if nb_personnes_a_charge > 0 else 0
    return premier_enfant + (zone_parameters.majoration_par_enf_supp * max(0, nb_personnes_a_charge - 1))

data_parametres_avec_personne_a_charge = {
    ('Personne seule ou couple', '1'): [parametres_locatif_zone_1.un_enfant, parametres_locatif_zone_2.un_enfant, parametres_locatif_zone_3.un_enfant],
    ('Personne seule ou couple', '2'): [get_plafond_loyer(parametres_locatif_zone_1, 2), get_plafond_loyer(parametres_locatif_zone_2, 2), get_plafond_loyer(parametres_locatif_zone_3, 2)],
    ('Personne seule ou couple', '3'): [get_plafond_loyer(parametres_locatif_zone_1, 3), get_plafond_loyer(parametres_locatif_zone_2, 3), get_plafond_loyer(parametres_locatif_zone_3, 3)],
    ('Personne seule ou couple', '4'): [get_plafond_loyer(parametres_locatif_zone_1, 4), get_plafond_loyer(parametres_locatif_zone_2, 4), get_plafond_loyer(parametres_locatif_zone_3, 4)],
    ('Personne seule ou couple', '5'): [get_plafond_loyer(parametres_locatif_zone_1, 5), get_plafond_loyer(parametres_locatif_zone_2, 5), get_plafond_loyer(parametres_locatif_zone_3, 5)],
    ('', 'Par pàc sup.'): [
        f'+ {parametres_locatif_zone_1.majoration_par_enf_supp}',
        f'+ {parametres_locatif_zone_2.majoration_par_enf_supp}',
        f'+ {parametres_locatif_zone_3.majoration_par_enf_supp}'
        ]
}

index = Index([id_zone_1, id_zone_2, id_zone_3], name='Zone')
df_parametres_avec_personne_a_charge = DataFrame(data_parametres_avec_personne_a_charge, index=index)

# Réorganiser les colonnes pour correspondre à l'ordre souhaité
df_parametres_avec_personne_a_charge = df_parametres_avec_personne_a_charge[[('Personne seule ou couple', '1'), ('Personne seule ou couple', '2'),
         ('Personne seule ou couple', '3'), ('Personne seule ou couple', '4'),
         ('Personne seule ou couple', '5'), ('', 'Par pàc sup.')]]

df_parametres_avec_personne_a_charge.columns.names = [None, 'Nombre de personnes à charge (pàc)']

In [ ]:
from utils_calculate import get_latest_parameter_references

if (~logement_chambre and ~coloc):  # TODO ajouter hors sous-location
    print(f"Le ménage a-t-il des personnes à charge ? {au_moins_une_personne_a_charge}")

    if au_moins_une_personne_a_charge:
        plafond_loyer_applicable = get_plafond_loyer(plafond_loyer_period_zone, al_nb_personnes_a_charge)
        loyer_plafond_toute_situation = plafond_loyer_applicable

        if (al_nb_personnes_a_charge > 5):
            print(f"Le ménage a plus de 5 personnes à charge : {al_nb_personnes_a_charge}")
        print(f"En zone '{id_zone_apl_applicable}', le plafond de loyer qui lui est applicable est : {plafond_loyer_applicable} €")
        df_parametres_avec_personne_a_charge.style.format(precision=NB_DECIMALES).map(colorie_reference, reference=plafond_loyer_applicable)
    else:
        plafond_loyer_applicable = plafond_loyer_period_zone['personnes_seules' if celibataire else 'couples']
        loyer_plafond_toute_situation = plafond_loyer_applicable

        print(f"En zone '{id_zone_apl_applicable}', le plafond de loyer qui lui est applicable est : {plafond_loyer_applicable} €")
        df_parametres_sans_personne_a_charge.style.format(precision=NB_DECIMALES).map(colorie_reference, reference=plafond_loyer_applicable)

    print("Quelle est la référence législative à l'origine de ces valeurs ?")
    parameter_all_periods = tax_benefit_system.parameters.prestations_sociales.aides_logement.allocations_logement.locatif.formule.l_plafonds_loyers.par_zone.children[id_zone_apl_applicable].un_enfant
    get_latest_parameter_references(parameter_all_periods)
    # exemple d'arrêté disponible sur tricoteuses : https://legal.tricoteuses.fr/api/recherche?latest=true&q=JORFARTI000050278404


Le ménage a-t-il des personnes à charge ? False
En zone 'zone_1', le plafond de loyer qui lui est applicable est : 329.71 €


,Personne seule,Couple
Zone,,
zone_1,329.71,397.64
zone_2,287.35,351.72
zone_3,269.32,326.48


Quelle est la référence législative à l'origine de ces valeurs ?


[{'title': 'Arrêté du 27/09/2024, art. 1',
  'href': 'https://www.legifrance.gouv.fr/jorf/article_jo/JORFARTI000050278404'},
 {'title': 'Arrêté du 27/09/2019, art. 7',
  'href': 'https://www.legifrance.gouv.fr/loda/id/JORFTEXT000039160329/2024-09-29/'}]

In [97]:
from numpy import round

L_analyse = min(loyer, loyer_plafond_toute_situation)
condition_verification_L = (round(L_calcule, NB_DECIMALES) - L_analyse) <= 0.001  # marge d'erreur pour écarts numpy.float32 et float (ici largement supérieure)

print(f"📍 Le loyer retenu doit être le minimum entre le loyer du logement {loyer} et le loyer plafond {loyer_plafond_toute_situation}, soit : {L_analyse}")
affiche_resultat("L", "Le loyer calculé pour l'APL correspond-t-il bien à l'attendu ?", condition_verification_L, round(L_calcule, NB_DECIMALES))

📍 Le loyer retenu doit être le minimum entre le loyer du logement 700.0 et le loyer plafond 329.71, soit : 329.71
✅ Le loyer calculé pour l'APL correspond-t-il bien à l'attendu ?
L : 329.71


## C - Forfait charges pris en compte

In [98]:
aide_logement_charges = simulation.calculate('aide_logement_charges', period)[0]
C_calcule = aide_logement_charges

forfait_charges_toute_situation = None

# paramètres forfait charges - cas général
montant_forfaitaire_charges_seul_ou_couple = parametres_secteur_locatif_period.formule.c_forfait_charges.cas_general.cas_general
montant_majoration_charges_par_enfant = parametres_secteur_locatif_period.formule.c_forfait_charges.cas_general.majoration_par_enfant

In [99]:
# TABLEAU SANS PERSONNE À CHARGE - CAS GÉNÉRAL
# TODO cas des colocataires

if (~au_moins_une_personne_a_charge):
    fofait_charges_applicable = montant_forfaitaire_charges_seul_ou_couple
    forfait_charges_toute_situation = fofait_charges_applicable

    data_parametres_charges_sans_personne_a_charge = {
        'Personne seule ou en couple': [montant_forfaitaire_charges_seul_ou_couple]
    }

    index = Index(['Cas général'], name='Cas')
    df_parametres_charges_sans_personne_a_charge = DataFrame(data_parametres_charges_sans_personne_a_charge, index=index)
    df_parametres_charges_sans_personne_a_charge.style.format(precision=NB_DECIMALES).map(colorie_reference, reference=fofait_charges_applicable)

,Personne seule ou en couple
Cas,
Cas général,59.97


In [ ]:
# TABLEAU AVEC PERSONNE À CHARGE
# TODO cas des colocataires

if au_moins_une_personne_a_charge:
    fofait_charges_applicable = montant_forfaitaire_charges_seul_ou_couple + al_nb_personnes_a_charge * montant_majoration_charges_par_enfant
    forfait_charges_toute_situation = fofait_charges_applicable

    data_parametres_charges_avec_personne_a_charge = {
        ('Personne seule ou couple', '1'): [montant_forfaitaire_charges_seul_ou_couple + montant_majoration_charges_par_enfant],
        ('Personne seule ou couple', '2'): [montant_forfaitaire_charges_seul_ou_couple + 2 * montant_majoration_charges_par_enfant],
        ('Personne seule ou couple', '3'): [montant_forfaitaire_charges_seul_ou_couple + 3 * montant_majoration_charges_par_enfant],
        ('', 'Par pàc sup.'): [
            f'+ {montant_majoration_charges_par_enfant}'
            ]
    }

    index = Index(['Cas général'], name='Cas')
    df_parametres_charges_avec_personne_a_charge = DataFrame(data_parametres_charges_avec_personne_a_charge, index=index)

    # Réorganiser les colonnes pour correspondre à l'ordre souhaité
    df_parametres_charges_avec_personne_a_charge = df_parametres_charges_avec_personne_a_charge[[
        ('Personne seule ou couple', '1'),
        ('Personne seule ou couple', '2'),
        ('Personne seule ou couple', '3'),
        ('', 'Par pàc sup.')]]

    df_parametres_charges_avec_personne_a_charge.columns.names = [None, 'Nombre de personnes à charge (pàc)']
    df_parametres_charges_avec_personne_a_charge.style.format(precision=NB_DECIMALES).map(colorie_reference, reference=fofait_charges_applicable)

In [101]:
C_analyse = forfait_charges_toute_situation
condition_verification_C = (round(C_calcule, NB_DECIMALES) - C_analyse) <= 0.001  # marge d'erreur pour écarts numpy.float32 et float (ici largement supérieure)

print(f"📍 Le fofait charges retenu dépend de la situation maritale '{label_situation_maritale}' et du nombre '{al_nb_personnes_a_charge}' de personnes à charges, soit : {C_analyse}")
affiche_resultat("C", "Le forfait charges calculé pour l'APL correspond-t-il bien à l'attendu ?", condition_verification_C, round(C_calcule, NB_DECIMALES))

📍 Le fofait charges retenu dépend de la situation maritale 'célibataire' et du nombre '0' de personnes à charges, soit : 59.97
✅ Le forfait charges calculé pour l'APL correspond-t-il bien à l'attendu ?
C : 59.97


## P0 - Participation personnelle minimale

```
P0 est une composante de la participation personnelle Pp qui correspond à la part fixe, indépendante du revenu, que doit au minimum acquitter le locataire.
```

In [109]:
p0_taux = parametres_secteur_locatif_period.formule.pp_particip_perso.p0_particip_min.p0_taux
p0_forfait = parametres_secteur_locatif_period.formule.pp_particip_perso.p0_particip_min.p0_forfait

depense_logement_plafonnee_calculee = L_calcule + C_calcule
depense_logement_plafonnee_analyse = L_analyse + C_analyse

p0_analyse = max(p0_forfait, p0_taux * depense_logement_plafonnee_analyse)
p0_calcule = max(p0_forfait, p0_taux * depense_logement_plafonnee_calculee)  # P0 est inclue dans aide_logement_participation_personnelle
condition_verification_P0 = (round(p0_calcule, NB_DECIMALES) - p0_analyse) <= 0.001  # marge d'erreur pour écarts numpy.float32 et float (ici largement supérieure)

print("📍 La participation personnelle minimale doit être la plus élevée des deux valeurs suivantes :")
print(f"* {p0_taux * 100} % de la dépense de logement plafonnée (L + C = {L_analyse} + {C_analyse}) soit : {round(p0_taux * depense_logement_plafonnee_analyse, NB_DECIMALES)}")
print(f"* {p0_forfait} € (montant forfaitaire) \n")
affiche_resultat("P0", "La participation personnelle minimale calculée dont doit s'acquitter le locataire correspond-t-elle bien à l'attendu ?", condition_verification_P0, round(p0_calcule, NB_DECIMALES))

📍 La participation personnelle minimale doit être la plus élevée des deux valeurs suivantes :
* 8.5 % de la dépense de logement plafonnée (L + C = 329.71 + 59.97) soit : 33.12
* 39.15 € (montant forfaitaire) 

✅ La participation personnelle minimale calculée dont doit s'acquitter le locataire correspond-t-elle bien à l'attendu ?
P0 : 39.15
